In [1]:
import torch 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
torch.manual_seed(1)
lin = nn.Linear(5, 3)
data = Variable(torch.randn(10, 5)) # create a 10 * 5 matrix of random vars

In [3]:
lin(data) # an affine map

Variable containing:
 0.1755 -0.3268 -0.5069
-0.6602  0.2260  0.1089
-0.4935  0.5282  0.1833
-0.0047 -0.0912 -0.3908
-1.0152 -0.6522 -0.5606
-1.2472  0.0609  0.2764
 0.3908  0.0782 -0.1344
-0.4794  0.5238  0.3645
-0.5699 -0.1749 -0.2400
-0.8788 -0.0109  0.4711
[torch.FloatTensor of size 10x3]

In [4]:
print(F.relu(lin(data)))

Variable containing:
 0.1755  0.0000  0.0000
 0.0000  0.2260  0.1089
 0.0000  0.5282  0.1833
 0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000
 0.0000  0.0609  0.2764
 0.3908  0.0782  0.0000
 0.0000  0.5238  0.3645
 0.0000  0.0000  0.0000
 0.0000  0.0000  0.4711
[torch.FloatTensor of size 10x3]



In [5]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE)

26


In [11]:
class BOWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BOWClassifier, self).__init__()
        self.lin = nn.Linear(vocab_size, num_labels)
    def forward(self, x):
        return F.softmax(self.lin(x))

def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            raise ValueError('houston we have a problem')
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE)

    

In [12]:
for param in bow.parameters():
    print(param)
    

Parameter containing:

Columns 0 to 9 
-0.0449  0.1427  0.1553  0.1855 -0.0398 -0.1524  0.1931 -0.0418 -0.0807  0.0478
 0.0088  0.0875  0.0235 -0.0982  0.1131  0.1206 -0.0114 -0.0241  0.1782  0.1714

Columns 10 to 19 
-0.1371  0.1289  0.1229 -0.1556 -0.1611 -0.0172  0.0824 -0.0057 -0.0994  0.0045
-0.1112  0.1919  0.0485 -0.1303  0.1074 -0.1464  0.1812 -0.1261  0.0555  0.0597

Columns 20 to 25 
-0.1843 -0.1386 -0.1306  0.1615  0.1729 -0.0666
 0.0466  0.1627 -0.0815 -0.0828 -0.1699 -0.0080
[torch.FloatTensor of size 2x26]

Parameter containing:
1.00000e-02 *
 -9.2904
  0.7858
[torch.FloatTensor of size 2]



In [13]:
sample_data, sample_label = data[0]
bow_vector = torch.autograd.Variable(make_bow_vector(sample_data, word_to_ix))
logprobs = bow(bow_vector)
print(logprobs)

Variable containing:
 0.4726  0.5274
[torch.FloatTensor of size 1x2]



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [14]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}
ix_to_label = {v: k for k, v in label_to_ix.items()}
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    print('prediction: {}'.format(ix_to_label[pred]))
    print('actual: {}'.format(label))

    
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

# the training loop
for epoch in range(100):
    for instance, label in data:
        # get the training data
        bow.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix))
        label = Variable(make_target(label, label_to_ix))
        probs = bow(bow_vec) # forward pass
        loss = loss_function(probs, label)
        loss.backward()
#        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()

print('--- AFTER TRAINING ---')
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = bow(bow_vec)
    print(logprobs)
    pred = np.argmax(logprobs.data.numpy())
    print('prediction: {}'.format(ix_to_label[pred]))
    print('actual: {}'.format(label))



Variable containing:
 0.5981  0.4019
[torch.FloatTensor of size 1x2]

prediction: SPANISH
actual: SPANISH
Variable containing:
 0.3473  0.6527
[torch.FloatTensor of size 1x2]

prediction: ENGLISH
actual: ENGLISH
--- AFTER TRAINING ---
Variable containing:
 0.9168  0.0832
[torch.FloatTensor of size 1x2]

prediction: SPANISH
actual: SPANISH
Variable containing:
 0.0525  0.9475
[torch.FloatTensor of size 1x2]

prediction: ENGLISH
actual: ENGLISH


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
